In [1]:
#!pip3 install openpyxl

In [2]:
import pandas as pd
import numpy as np
import warnings
from openpyxl import load_workbook
import re
import folium
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

In [3]:
def get_cell_info(wb, data_sheet, row, col):    
    info = data_sheet.cell(row+1, col+1)
    info_crp = {'bold': 1*(info.font.bold),
                    'italic': 1*(info.font.italic),
                    'locked': info.protection.locked,
                    'indent': info.alignment.indent,
                    'value': info.value
                   }
        
    return info_crp

In [67]:
def get_data(wb, data_sheet_nm, label_col, data_cols, columns_):
    
    data_sheet = wb.get_sheet_by_name(data_sheet_nm)
    freezed_row = int(data_sheet.freeze_panes[1]) - 1
    
    data_clean = []
    indent = 0
    r = ''

    end_row = freezed_row  
    
    while True:
        try:
            row_ = ['']*(3 + len(data_cols))
            cell_reg_name = get_cell_info(wb, data_sheet,
                          end_row, label_col)
            cell_population = [get_cell_info(wb, data_sheet, 
                          end_row, i) for i in data_cols]
            
            indent = cell_reg_name['indent']
            #Получаем название региона 
            if indent == 0:
                if (cell_reg_name['bold']==1):
                    r = cell_reg_name['value']
                    row_[0] = 0
                    row_[1] = r
                    row_[2] = r
                    for n,i in enumerate(cell_population):
                        row_[n+3] = i['value']
                else:
                    m = cell_reg_name['value']
                    row_[0] = 1
                    row_[1] = r
                    row_[2] = m
                    for n,i in enumerate(cell_population):
                        row_[n+3] = i['value']
                if (row_[-1] != '')&(row_[-1] != np.nan)&(row_[-1] is not None):
                    data_clean.append(row_) 
                else:
                    break
            end_row+=1
        except Exception as e:
            print(e)
            break
    result = pd.DataFrame(data_clean)
    columns = ['level', 'group', 'region']
    columns.extend(columns_)
    result.columns = columns        
    return result

In [68]:
def clean_names(nm):
    return re.sub(r'[^\w\s]','',(re.sub(' +', ' ', nm).strip()))

In [69]:
def clean_values(val):
    if (re.fullmatch('([0-9]*)|([0-9]*\.[0-9]*)', str(val)) is not None):
        val = float(val)
    else:
        val = 0
    return val

In [70]:
years = [str(i) for i in range(2000, 2021)] #доступны данные за этот период
label_col_idx = 0 #номер столбца с названием региона
#data_col_idx = [years.index(str(2020)) + 1] #данные за определенный год
data_col_idxs = [i+1 for i in range(len(years))] #данные за весь период

# Данные по населению

In [71]:
wb = load_workbook(filename = './data/2021/Раздел 1 - Население.xlsx')
info = pd.read_excel('./data/2021/Раздел 1 - Население.xlsx')
info.head(3)

,Unnamed: 0,Unnamed: 1
0,Социально-экономические показатели по субъекта...,NaN
1,1. НАСЕЛЕНИЕ,NaN
2,1.1.,Численность населения


In [72]:
df_population = get_data(wb, '1.1.', label_col_idx, data_col_idxs, years)
print(df_population.shape[0])
df_population.head()

90


,level,group,region,2000,2001,2002,2003,2004,2005,2006,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,Центральный федеральный округ,Центральный федеральный округ,38175.1,38068.5,37946.8,37965.1,38044,38109,38183.4,...,38537.6,38678.9,38819.9,38951.5,39104.3,39209.6,39311.4,39378.1,39433.6,39251.0
1,1,Центральный федеральный округ,Белгородская область,1507.0,1508.1,1511.9,1513.9,1511.7,1511.7,1514.2,...,1536.1,1541,1544.1,1547.9,1550.1,1552.9,1549.9,1547.4,1549.2,1541.3
2,1,Центральный федеральный округ,Брянская область,1407.9,1391.4,1375.0,1360.2,1344.1,1327.7,1312.7,...,1264.4,1253.6,1242.6,1233,1225.8,1220.5,1211,1200.2,1192.5,1182.7
3,1,Центральный федеральный округ,Владимирская область,1558.0,1539.2,1520.1,1509.6,1497.6,1486.5,1475.9,...,1431.9,1421.7,1413.3,1405.6,1397.2,1389.6,1378.3,1365.8,1358.4,1342.1
4,1,Центральный федеральный округ,Воронежская область,2422.4,2397.1,2374.4,2367.4,2364.9,2360.9,2353.8,...,2331.5,2330.4,2328.9,2331.1,2333.5,2335.4,2333.8,2327.8,2324.2,2305.6


In [74]:
df_population['group'] = df_population['group'].map(clean_names)
df_population['region'] = df_population['region'].map(clean_names)
df_population[years] = df_population[years].apply(lambda x: x.fillna(0).map(clean_values))

# Данные о животных

In [75]:
wb = load_workbook(filename = './data/2021/Раздел 13 -Сельское хозяйство.xlsx')
info = pd.read_excel('./data/2021/Раздел 13 -Сельское хозяйство.xlsx')
info[info['Unnamed: 1'].isin(['Поголовье крупного рогатого скота', 'Поголовье свиней',
       'Поголовье овец и коз'])]['Unnamed: 0'].unique()

array(['13.25.', '13.26.', '13.27.'], dtype=object)

In [92]:
df_list = []
for (n, sheet_nm) in enumerate(info[info['Unnamed: 1'].isin(['Поголовье крупного рогатого скота', 'Поголовье свиней',
       'Поголовье овец и коз'])]['Unnamed: 0'].unique()):
    df = get_data(wb, sheet_nm, label_col_idx, data_col_idxs, years)
    df['group'] = df['group'].map(clean_names)
    df['region'] = df['region'].map(clean_names)
    df[years] = df[years].apply(lambda x: x.fillna(0).map(clean_values))
    df_list.append(df)
    
df_krs = df_list[0]
df_pigs = df_list[1]
df_sheeps = df_list[2]

# На душу населения

Людей и животных Росстат считает примерно похожим образом: численность населения измеряется в тысячах человек; поголовье КРС, свиней и овец/коз - в тысячах голов.

## Расчёт

In [93]:
df_final = df_population\
    .merge(df_krs, on = ['level', 'group', 'region'], suffixes = ['', '_krs'], how = 'left')\
    .merge(df_pigs, on = ['level', 'group', 'region'], suffixes = ['', '_pigs'], how = 'left')\
    .merge(df_sheeps, on = ['level', 'group', 'region'], suffixes = ['', '_sheeps'], how = 'left')

In [94]:
df_final = df_final[df_final['level']==1]

In [95]:
for y in years:
    n = y + '_animals'
    df_final[n] = df_final[y+'_krs'] + df_final[y+'_pigs'] + df_final[y+'_sheeps']

In [96]:
krs_cols = [y + '_krs' for y in years]
pigs_cols = [y + '_pigs' for y in years]
sheeps_cols = [y + '_sheeps' for y in years]
animal_cols = [y + '_animals' for y in years]

In [97]:
df_final.set_index(['level', 'group', 'region'], inplace = True)

In [98]:
krs_shares = np.divide(*df_final[krs_cols].align(df_final[years], axis=0))
krs_shares.columns = [i + '_pc' for i in krs_cols]

pigs_shares = np.divide(*df_final[pigs_cols].align(df_final[years], axis=0))
pigs_shares.columns = [i + '_pc' for i in pigs_cols]

sheeps_shares = np.divide(*df_final[sheeps_cols].align(df_final[years], axis=0))
sheeps_shares.columns = [i + '_pc' for i in sheeps_cols]

animals_shares = np.divide(*df_final[animal_cols].align(df_final[years], axis=0))
animals_shares.columns = [i + '_pc' for i in animal_cols]

## Результат

### На 2020 год

In [99]:
krs_shares[krs_shares['2020_krs_pc']>=1][['2020_krs_pc']].sort_values('2020_krs_pc', ascending = False)

,,,2020_krs_pc
level,group,region,
1,Южный федеральный округ,Республика Калмыкия,1.332222


In [100]:
pigs_shares[pigs_shares['2020_pigs_pc']>=1][['2020_pigs_pc']].sort_values('2020_pigs_pc', ascending = False)

2020_pigs_pc
level group                            region                            
1     Центральный федеральный округ    Белгородская область      2.954389
      СевероЗападный федеральный округ Псковская область         2.328120
      Центральный федеральный округ    Курская область           2.060374
                                       Тамбовская область        1.174678

In [101]:
sheeps_shares[sheeps_shares['2020_sheeps_pc']>=1][['2020_sheeps_pc']].sort_values('2020_sheeps_pc', ascending = False)

2020_sheeps_pc
level group                              region                                        
1     Южный федеральный округ            Республика Калмыкия                   7.160370
      Сибирский федеральный округ        Республика Тыва                       3.698245
      СевероКавказский федеральный округ КарачаевоЧеркесская Республика        2.384401
      Сибирский федеральный округ        Республика Алтай                      1.715385
      СевероКавказский федеральный округ Республика Дагестан                   1.446973
      Южный федеральный округ            Астраханская область                  1.281820

In [102]:
animals_shares[animals_shares['2020_animals_pc']>=1][['2020_animals_pc']].sort_values('2020_animals_pc', ascending = False)

2020_animals_pc
level group                              region                                         
1     Южный федеральный округ            Республика Калмыкия                    8.529630
      Сибирский федеральный округ        Республика Тыва                        4.299031
      Центральный федеральный округ      Белгородская область                   3.151625
      СевероКавказский федеральный округ КарачаевоЧеркесская Республика         2.718522
      Сибирский федеральный округ        Республика Алтай                       2.664253
      СевероЗападный федеральный округ   Псковская область                      2.486295
      Центральный федеральный округ      Курская область                        2.338532
      СевероКавказский федеральный округ Республика Дагестан                    1.745316
      Южный федеральный округ            Астраханская область                   1.578874
      Центральный федеральный округ      Тамбовская область                     1.332663
                                         Орловская область                      1.161446
      Приволжский федеральный округ      Республика Мордовия                    1.092683
      Сибирский федеральный округ        Республика Хакасия                     1.003759

# Карта

In [103]:
geolocator = Nominatim(user_agent="tolko sprosit")

In [104]:
list_of_regions = [i[2] for (i, n) in pd.DataFrame([animals_shares[i]>=1 for i in animals_shares.columns])\
                        .sum(axis = 0)\
                      .items() if n>0]

In [105]:
coordinates = {}
for r in tqdm(list_of_regions):
    #a bit of hardcode
    if r=='КарачаевоЧеркесская Республика':
        r_fixed = 'Карачаево - Черкесская Республика'
        coordinates.update({r: [geolocator.geocode(r_fixed).latitude, \
                                       geolocator.geocode(r_fixed).longitude]})
    else:
        coordinates.update({r: [geolocator.geocode(r).latitude, \
                                       geolocator.geocode(r).longitude]})

In [43]:
year = 2020

In [51]:
russia_map = folium.Map(
    location = [geolocator.geocode('Тюмень').latitude, geolocator.geocode('Тюмень').longitude], 
    tiles='cartodbpositron',
    zoom_start = 3.3)

for r in coordinates.keys():
    cow_icon = folium.features.CustomIcon('https://emojio.ru/images/apple-b/1f42e.png',
                                          icon_size=(40, 40))
    pig_icon = folium.features.CustomIcon('https://emojio.ru/images/apple-b/1f437.png',
                                          icon_size=(40, 40))
    sheep_icon = folium.features.CustomIcon('https://emojio.ru/images/apple-b/1f40f.png',
                                          icon_size=(40, 40))
    animals_icon = folium.features.CustomIcon('https://emojio.ru/images/apple-b/1f984.png',
                                          icon_size=(40, 40))
    if r in krs_shares[krs_shares[str(year) + '_krs_pc']>=1].reset_index()['region'].values:
        icon_ = cow_icon
        nm = 'КРС'
        vv = krs_shares.reset_index()[krs_shares.reset_index()['region']==r][str(year) + '_krs_pc'].values[0]
    elif r in pigs_shares[pigs_shares[str(year) + '_pigs_pc']>=1].reset_index()['region'].values:
        icon_ = pig_icon
        nm = 'свиней'
        vv = pigs_shares.reset_index()[pigs_shares.reset_index()['region']==r][str(year) + '_pigs_pc'].values[0]
    elif r in sheeps_shares[sheeps_shares[str(year) + '_sheeps_pc']>=1].reset_index()['region'].values:
        icon_ = sheep_icon
        nm = 'овец и коз'
        vv = sheeps_shares.reset_index()[sheeps_shares.reset_index()['region']==r][str(year) + '_sheeps_pc'].values[0]
    elif r in animals_shares[animals_shares[str(year) + '_animals_pc']>=1].reset_index()['region'].values:
        icon_ = animals_icon
        nm = 'с/х животных'
        vv = animals_shares.reset_index()[animals_shares.reset_index()['region']==r][str(year) + '_animals_pc'].values[0]
    else:
        icon_ = ''
    
    if icon_ != '':
        folium.Marker(coordinates[r],
                  icon=icon_,
                  tooltip=r,
                  popup='<b>{}</b> {} голов {} на человека'.format(r,round(vv, 2), nm)
                 ).add_to(russia_map)
    
russia_map

In [53]:
russia_map.save(str(year) + '_livestock.html')